In [ ]:
# LIBRARIES IMPORT
import mercury as mr
import ee
import eefolium as geemap
import os
import webbrowser
import logging
import folium
from ipyleaflet import json
import matplotlib.pyplot as plt
from colorama import Fore
from PIL import Image
import concurrent.futures
import shutil
import sys

# PROGRAM MODULES IMPORT
sys.path.insert(0, 'gee//modules//') # gee modules path
sys.path.insert(1, 'osm//') # osm modules path
sys.path.insert(2, 'common//') # common files 
import create_ee_creds as cec
import txtreading as txtr
import landsat_init as l_init
import sentinel_init as s_init
import modis_init as m_init
import othersds_init as o_init
import customds_init as cd_init
import osm_processing as op
import welcome_processing as wp
import plugins_handler as ph

In [ ]:
# PROGRAM PRE-PROCESSING

logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
handler = logging.FileHandler('gee//gaia-gee-errors.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎 Not a final program solution, something might be changed 🌎 Current version - 0.63b 🌏**')
sources_list = ["GAIA HUB", "Combined Cartography and Satellite Imagery Module"]
control_panel_choice = mr.Select(value="GAIA HUB", choices=sources_list, label="Tabs 🔍:")

app  = mr.App(title="GAIA v0.63b [BETA] 🌐",
              description="Open-Source Geospatial & Aerial Images Analyser",
              show_code=False,
              show_prompt=False,
              continuous_update=True,
              static_notebook=False,
              show_sidebar=True,
              full_screen=True,
              allow_download=False)

In [ ]:
if control_panel_choice.value == "GAIA HUB":
    current_directory = os.getcwd()
    cache_path = current_directory + "\cache"
    logs_path = current_directory + "\gee\gaia-gee-errors.log"    
    ee_reg = mr.Button(label="Create GEE account")
    ee_service_acc = mr.Button(label="Create & Manage GEE service account")
    clr_cache_and_logs = mr.Button(label="Clear GAIA cache folder")
    gaia_wiki = mr.Button(label="Open GAIA wiki")
    project_page = mr.Button(label="Open GitHub project's page")
    current_settings = mr.Checkbox(value=False, label='Show current settings')

    mr.Markdown('**PROGRAM DETAILS**')
    print(Fore.GREEN + "Current GAIA directory path: {}".format(current_directory))
    print(Fore.GREEN + f"The size of the logs file is {os.path.getsize(logs_path) / (1024 * 1024)} MB.")
    print(Fore.GREEN + f"The size of the cache folder is {wp.get_folder_size(cache_path) / (1024 * 1024)} MB.")

    if ee_reg.clicked:
        webbrowser.open_new_tab('https://code.earthengine.google.com/register')

    if gaia_wiki.clicked:
        webbrowser.open_new_tab('https://github.com/OSINT-TECHNOLOGIES/gaia/wiki')
        
    if project_page.clicked:
        webbrowser.open_new_tab('https://github.com/OSINT-TECHNOLOGIES/gaia')
        
    if ee_service_acc.clicked:
        webbrowser.open_new_tab('https://console.cloud.google.com/iam-admin/serviceaccounts/')

    if clr_cache_and_logs.clicked:
        try:
            shutil.rmtree(cache_path)
        except FileNotFoundError:
            print(Fore.RED + "Sorry, but cache folder was not found in GAIA directory. You can track this folder's volume in the line above")

    txtr.ee_guide_read()
    print('\n'*2)
    
    if current_settings.value:
        txtr.settings_read()
        settings_redactor = mr.Checkbox(value=False, label='Open settings redactor [work in progress]')

In [ ]:
# OSM+GEE PROCESSING

if control_panel_choice.value == "Combined Cartography and Satellite Imagery Module":
    if str((cec.service_acc_filename).rstrip()) not in os.listdir():
        print(Fore.RED + '[GEE file/directory missing error]. Your EE credentials .JSON file was not found in GAIA directory')
        mr.Stop()
    else:
        credentials = ee.ServiceAccountCredentials((cec.service_acc_mail).rstrip(), (cec.service_acc_filename).rstrip())
        ee.Initialize(credentials)

    geojson_file = mr.File(label="Upload GeoJSON file for processing", max_file_size="100MB")
    ds_info = mr.Checkbox(value=False, label='Show datasets knowledge base')

    date_one = mr.Text(value="2022-01-01", label="Enter start date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter final date (format YYYY-MM-DD)", rows=1)

    max_cloud_covering = mr.Slider(value=80, label='Max cloud cover (%, may affect datasets quality)', min=0, max=100)

    selected_datasets = mr.MultiSelect(label="Select datasets",
                              value=[],
                              choices=["All Landsat datasets", "All Sentinel datasets", "All MODIS datasets", "All other datasets", "Landsat 7 SR", "Landsat 8 SR", "Landsat 8 TOA",
                                       "Landsat 9 RAW", "Landsat 9 TOA", "Landsat 9 SR", "Sentinel 2 MSI TOA", "Sentinel 2 MSI SR", "Sentinel 5P Cloud",
                                       "MODIS Snowcover", "MODIS TSR", "MODIS TTA&FD", "ALOS TD", "NASA FIRMS"])
    
    manual_input = mr.Checkbox(value=False, label='Open custom GEE dataset menu')
    add_datasets = mr.Checkbox(value=False, label="View additional cartography maps")
    mini_map = mr.Checkbox(value=False, label='Show mini map')
    more_basemaps = mr.Checkbox(value=False, label='Show additional basemaps')
    terminator = mr.Checkbox(value=False, label='Show day/night separation')

    m = geemap.Map()
    ph.basemaps_init(m)

    if more_basemaps.value:
        ph.additional_basemaps_init(m)  
    
    if mini_map.value:
        ph.minimap_init(m)

    if terminator.value:
        ph.terminator_init(m)

    if geojson_file.filepath is not None:
        try:
            geojson = open(geojson_file.filepath)
            geojson_info = json.load(geojson)
            coordinates = geojson_info['features'][0]['geometry']['coordinates']
            folium.GeoJson(geojson_info, name='geojson').add_to(m)
        except IndexError as error:
            print(Fore.RED + '[OSM GeoJSON file error] - {}'.format(str(error)))
    else:
        pass

    if add_datasets.value:
        op.additional_datasets(m)
        op.map_layers(m)

    landsat = l_init.LandsatInitialization(m, date_one, date_two, max_cloud_covering)
    sentinel = s_init.SentinelInitialization(m, date_one, date_two, max_cloud_covering)
    modis = m_init.ModisInitialization(m, date_one, date_two, max_cloud_covering)
    others = o_init.OthersInitialization(m, date_one, date_two)

    try:
        if "All Landsat datasets" in selected_datasets.value:
            with concurrent.futures.ThreadPoolExecutor() as landsat_caller:
                landsat_caller.submit(landsat.ls7sr_init)
                landsat_caller.submit(landsat.ls8toa_init)
                landsat_caller.submit(landsat.ls8rawtc_init)
                landsat_caller.submit(landsat.ls8sr_init)
                landsat_caller.submit(landsat.ls9_init)
                landsat_caller.submit(landsat.ls9rawtc_init)
                landsat_caller.submit(landsat.ls9toatc_init)
    
        if "All Sentinel datasets" in selected_datasets.value:
            with concurrent.futures.ThreadPoolExecutor() as sentinel_caller:
                sentinel_caller.submit(sentinel.sen2msisr_init)
                sentinel_caller.submit(sentinel.sen2msitoa_init)
                sentinel_caller.submit(sentinel.sen5pc_init)

        if "All MODIS datasets" in selected_datasets.value:
            with concurrent.futures.ThreadPoolExecutor() as modis_caller:
                modis_caller.submit(modis.modissc_init)
                modis_caller.submit(modis.modistsr_init)
                modis_caller.submit(modis.modistta_init)

        if "All other datasets" in selected_datasets.value: 
            with concurrent.futures.ThreadPoolExecutor() as others_caller:
                others_caller.submit(others.firms_init)
                others_caller.submit(others.alostd_init)

        if "Landsat 7 SR" in selected_datasets.value:
            landsat.ls7sr_init()
        if "Landsat 8 TOA" in selected_datasets.value:
            landsat.ls8toa_init()
        if "Landsat 8 RAW" in selected_datasets.value:
            landsat.ls8rawtc_init()
        if "Landsat 8 SR" in selected_datasets.value:
            landsat.ls8sr_init()
        if "Landsat 9 SR" in selected_datasets.value:
            landsat.ls9_init()
        if "Landsat 9 RAW" in selected_datasets.value:
            landsat.ls9rawtc_init()
        if "Landsat 9 TOA" in selected_datasets.value:
            landsat.ls9toatc_init()
        if "Sentinel 2 MSI SR" in selected_datasets.value:
            sentinel.sen2msisr_init()
        if "Sentinel 2 MSI TOA" in selected_datasets.value:
            sentinel.sen2msitoa_init()
        if "Sentinel 5P Cloud" in selected_datasets.value:
            sentinel.sen5pc_init()
        if "MODIS Snowcover" in selected_datasets.value:
            modis.modissc_init()
        if "MODIS TSR" in selected_datasets.value:
            modis.modistsr_init()
        if "ALOS TD" in selected_datasets.value:
            others.alostd_init()
        if "NASA FIRMS" in selected_datasets.value:
            others.firms_init()
        if "MODIS TTA&FD" in selected_datasets.value:
            modis.modistta_init()
            
    except ee.EEException as error:
        print(Fore.RED + '[GEE map plotting/initialization error] - {}'.format(str(error)))
        logger.error(error)
        pass

    if manual_input.value:
        try:
            cd_init.customds_initialization(m)
        except ee.EEException as error:
            print(Fore.RED + '[GEE map plotting/initialization error] - {}'.format(str(error)))
            logger.error(error)
            pass
        
    mr.Markdown(text = "#CARTOGRAPHY AND SATELLITE IMAGERY MODULE MAP 🌐")
    mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))
    
    if ds_info.value:
        mr.Markdown(text="**LANDSAT BANDS COMBINATIONS ⤵**")
        clb = Image.open('gee//data//clb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(clb)
        plt.axis('off')
        plt.show()
        mr.Markdown(text="**SENTINEL 2 BANDS COMBINATIONS ⤵**")
        csb = Image.open('gee//data//csb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(csb)
        plt.axis('off')
        plt.show()
        txtr.ds_info_read()
    
    ph.geocoder_init(m)
    ph.draw_init(m)
    m.addLayerControl()
    display(m)